# Experiments with Runnables

Create a few RunnableLambda, and compose them in sequence and in parallel

In [ ]:
from dotenv import load_dotenv

assert load_dotenv(verbose=True)

In [ ]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
    chain,
)

add_1 = RunnableLambda(lambda x: x + 1)
add_3 = RunnableLambda(lambda x: x + 3)


@chain
def mult_2(x: int):
    return x * 2


sequence = mult_2 | add_1
parallel = mult_2 | {"add_1": add_1, "add_3": add_3}

# same as above:
parallel1 = RunnableParallel(add_1=add_1, add_3=add_3)

add_1.invoke(5)  # 6

Run the runnable directly, batched, in parallel (multi-threaded whenever possible!), ...

In [ ]:
print(sequence.invoke(1))  # 3
sequence.batch([1, 2, 3])  # [3,5,7]
parallel.invoke(1)  # {'add_1': 3, 'add_3': 5}

await sequence.abatch([1, 2, 3, 4, 5])  # [3, 5, 7, 9, 11]

In [ ]:
parallel.invoke(1)  # {'add_1': 3, 'add_3': 5}

Display the Graph

In [ ]:
parallel.get_graph().print_ascii()

Use RunnablePassthrough

In [ ]:
runnable = add_1 | {"a": RunnablePassthrough(), "b": add_1}
runnable.invoke(10)  # {'a': 11, 'b': 12}

Demo 'bind' and 'RunnableConfig' : implement a filter, and log activities

In [ ]:
from langchain_core.runnables import RunnableConfig
from loguru import logger


@chain  # type: ignore
def max(x: int, max: int, min_x: int, config: RunnableConfig) -> int:
    log = config["configurable"]["logger"]
    if log:  # type: ignore
        log.info(f"check if {x} < {max}")
    return min(max, x)


a = sequence | max.bind(max=6, min_x=0)  # type: ignore
a.batch([1, 2, 3, 4, 5], config=({"logger": logger}))

Demo 'assign", that adds new fields to the dict output of the runnable and returns a new runnable. Often use with RunnablePassthrough to add a given argument to a dict.

In [ ]:
# Write a simpler example to illustrate .assign and RunnablePassthrough.assign AI!

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain_with_assign = RunnablePassthrough.assign(mult_10=lambda x: x["num"] * 10)

chain_with_assign.invoke({"num": 5})

Most of the time, there are several parameters, or it's a dictionary (almost equivalent in Python).
'itemgetter' create a function that can extract one or several fields from a dictionary. 

First have a look at how it works:

In [ ]:
from operator import itemgetter

dic = {
    "question": "What are the types of agent memory?",
    "generation": "The types of agent memory are: sensory memory, short-term memory, and long-term memory.",
    "documents": [],
}

getter_function = itemgetter("generation")
getter_function(dic)

here an example with Runnables

In [ ]:
adder = RunnableLambda(lambda d: d["op1"] + d["op2"])


mult_2_and_add = {"op1": itemgetter("a") | mult_2, "op2": itemgetter("b") | mult_2} | adder

mult_2_and_add.invoke({"a": 10, "b": 2})  # should return 2*10 + 2*2 = 24

Runnables can have fallback in case they break.

In [ ]:
@chain
def mult_10_fail(x: int):
    raise Exception("unavailable multiplication by 10 service")
    return x * 10


fallback_chain = mult_10_fail.with_fallbacks([mult_2])
fallback_chain.invoke(2)

See also : https://python.langchain.com/v0.2/docs/how_to/lcel_cheatsheet/